In [118]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle


In [119]:
data= pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [120]:
### Preprocess the Data 
### Lets drop the unwanted features

data.drop(['RowNumber','CustomerId','Surname'], axis=1,inplace =True)


In [121]:
## Encoding Categorical Features 

label_encoding_gender = LabelEncoder()
data['Gender']= label_encoding_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [122]:
### For geography I will go with One Hot Encoding to void ranking if we use Label Enconding
from sklearn.preprocessing import OneHotEncoder
onehotencoder=OneHotEncoder()
Geography_encoder=onehotencoder.fit_transform(data[['Geography']])
Geography_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [123]:
onehotencoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [124]:
data1=pd.DataFrame(Geography_encoder.toarray(),columns=onehotencoder.get_feature_names_out(['Geography']))
data1

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [125]:
data.head(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [126]:
### Lets combine the encoded categorical features to Data 
data=pd.concat([data.drop('Geography',axis=1),data1],axis=1)


In [127]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [128]:
with open('label_encoding_gender.pkl','wb') as file:
    pickle.dump(label_encoding_gender, file)

with open('onehotencoder.pkl','wb') as file:
    pickle.dump(onehotencoder, file)

In [129]:
### Divide the data into dependent and Independent split
X=data.drop('Exited',axis=1)
y=data['Exited']

### Split the data n training and test data

X_train,X_test,y_train,y_test=train_test_split(X,y, random_state=42,test_size=0.2)


##SCale the Features
Scaler=StandardScaler()

X_train=Scaler.fit_transform(X_train)
X_test=Scaler.transform(X_test)


In [130]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [145]:
with open('scaler.pkl','wb') as file:
    pickle.dump(Scaler,file)

## ANN IMPLEMENTATION


In [132]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [133]:
(X_train.shape[1],)

(12,)

In [134]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
    
])

In [135]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [136]:
import tensorflow
opt=tensorflow.keras.optimizers.legacy.Adam(learning_rate=0.01)

In [137]:
## Compile the model 

model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [138]:
### Set up the Tensorboard

log_dir= 'logs/fit' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [139]:
## Setup Early Stoppings

early_stopping_callback= EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [140]:
## Training The model

history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4094 - accuracy: 0.8255 - val_loss: 0.3599 - val_accuracy: 0.8505
Epoch 2/100
250/250 [==============================] - 0s 718us/step - loss: 0.3556 - accuracy: 0.8522 - val_loss: 0.3409 - val_accuracy: 0.8580
Epoch 3/100
250/250 [==============================] - 0s 703us/step - loss: 0.3458 - accuracy: 0.8576 - val_loss: 0.3580 - val_accuracy: 0.8530
Epoch 4/100
250/250 [==============================] - 0s 689us/step - loss: 0.3440 - accuracy: 0.8589 - val_loss: 0.3501 - val_accuracy: 0.8605
Epoch 5/100
250/250 [==============================] - 0s 701us/step - loss: 0.3388 - accuracy: 0.8625 - val_loss: 0.3421 - val_accuracy: 0.8610
Epoch 6/100
250/250 [==============================] - 0s 695us/step - loss: 0.3378 - accuracy: 0.8636 - val_loss: 0.3339 - val_accuracy: 0.8615
Epoch 7/100
250/250 [==============================] - 0s 695us/step - loss: 0.3360 - accuracy: 0.8618 - val_loss: 0.3356 - val_accu

In [141]:
model.save('model.h5')

/Users/sabareeshvangadari/END TO END ANN PROJECT/myvenv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [142]:
## Load Tensorboard Extension

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [143]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 14084), started 1:30:12 ago. (Use '!kill 14084' to kill it.)

In [144]:
### Load the Pickle File 